# **Projeto 2**

## **Questão 1)** Para a rede “Hamsterster”, calcule a média dos menores caminhos e o diâmetro.Use apenas o maior componente da rede e remova ciclos ou auto-conexões.

In [12]:
hamsterster = clean_graph(nx.read_edgelist('./data/hamsterster.txt', nodetype = int))

In [59]:
hamsterster_avarage_shortest_path = nx.average_shortest_path_length(hamsterster)

print(f'Média dos menores caminhos para a rede Hamsterster: {hamsterster_avarage_shortest_path:.2f}')

Média dos menores caminhos para a rede Hamsterster: 3.45


In [14]:
hamsterster_diameter = nx.diameter(hamsterster)

print(f'Diametro do maior componente da rede Hamsterster: {hamsterster_diameter}')

Diametro do maior componente da rede Hamsterster: 14


## **Questão 2)** Considere a rede “USairport500” e calcule a média e variância dos menores caminhos. Use apenas o maior componente da rede e remova ciclos ou autoconexões.

In [44]:
usairport = clean_graph(nx.read_edgelist('./data/usairport.txt', nodetype = int))

In [86]:
usairport_1_moment_shortest_path = momment_of_shortest_path_length_distribution(usairport, 1)
usairport_2_moment_shortest_path = momment_of_shortest_path_length_distribution(usairport, 2)

In [89]:
usairport_shortest_path_var = usairport_2_moment_shortest_path - usairport_1_moment_shortest_path**2

In [90]:
print(f'Média: {first_moment_shortest_path:.0f} e variância: {usairport_shortest_path_var:.0f} da rede Usairport')

Média: 3 e variância: 1 da rede Usairport


## **Questão 3)** Para a rede “USairport500”, calcule a entropia de Shannon da distribuição dos menores caminhos. Use logaritmo na base 2 e considere apenas o maior componente da rede.

In [91]:
usairport_shortest_path_shannon_entropy = shortest_path_length_shannon_entropy(usairport)

In [92]:
print(f'Shannon entropy: {usairport_shortest_path_shannon_entropy:.2f}')

Shannon entropy: 1.88


## **Questão 4)** Calcule o coeficiente de assortatividade da rede Advogato. Considere apenas o maior componente.

In [93]:
advogato = clean_graph(nx.read_edgelist('./data/advogato.txt', nodetype = int), False)

In [94]:
advogato_efficiency = nx.degree_assortativity_coefficient(advogato)

print(f'Network Global Efficiency: {advogato_efficiency}')

Network Global Efficiency: -0.08455192594199333


## **Questão 5)** Calcule o coeficiente de correlação de Pearson entre o grau médio dos vizinhos e o grau de cada vértice para a rede “word_adjacencies”. Isto é, entre k e knn(k). Use apenas o maior componente. Considere o exemplo da aula.

In [141]:
word_adjacencies = clean_graph(nx.read_edgelist('./data/word_adjacencies.txt', nodetype = int))

In [147]:
word_adjacencies_k, word_adjacencies_Pk = knnk(word_adjacencies)

In [148]:
pearson_ksknnk = np.corrcoef(word_adjacencies_k, word_adjacencies_Pk)[0, 1]

print(f'Pearson correlation: {pearson_ksknnk}')

Pearson correlation: -0.6753041480047248


## **Imports**

In [2]:
import numpy as np

In [3]:
import matplotlib.pyplot as plt

In [4]:
import networkx as nx

In [5]:
from scipy.stats import pearsonr, spearmanr

## **Settings**

In [6]:
%matplotlib inline

In [7]:
plt.rc('axes', titlesize=18)     # fontsize of the axes title
plt.rc('axes', labelsize=14)    # fontsize of the x and y labels
plt.rc('xtick', labelsize=13)    # fontsize of the tick labels
plt.rc('ytick', labelsize=13)    # fontsize of the tick labels
plt.rc('legend', fontsize=13)    # legend fontsize
plt.rc('font', size=13)          # controls default text sizes
plt.rc('figure', figsize = (8,8)) # Set the figure size 

## **Functions**

In [68]:
def shortest_path_length_shannon_entropy(G):
    
    _, Ppath = shortest_path_distribution(G)
    
    H = 0 
    
    for p in Ppath:
        
        if p > 0 :
            H -= p*np.log2(p)
    
    return H

In [74]:
def momment_of_shortest_path_length_distribution(G, m):
    
    V, P = shortest_path_distribution(G)
    
    M = np.sum((V**m)*P)
    
    return M

In [140]:
def shortest_path_distribution(G):
    
    shortest_paths = nx.shortest_path_length(G)
    
    shortest_paths_values = []
    
    for paths in shortest_paths:
    
        values = np.array(list(dict(paths[1]).values()))[1:]
    
        for value in values:
        
            shortest_paths_values.append(value)
    
    shortest_paths_values = np.array(shortest_paths_values)
    
    maxk = shortest_paths_values.max()
    mink = shortest_paths_values.min()
    
    paths_values = np.arange(maxk + 1)
    
    Ppath = np.zeros(maxk + 1)
    
    for path in shortest_paths_values:
        
        Ppath[path] += 1
        
    Ppath = Ppath/Ppath.sum()
    
    return paths_values, Ppath

In [146]:
def knnk(G):
    
    knnk_G = []
    ks_G = []
    
    knn_G = knn(G)
    
    vk = np.array(list(dict(G.degree).values()))
    
    for k in np.arange(vk.min(), vk.max()):
    
        aux = vk == k

        if len(knn_G[aux]) > 0 :

            average_knn = knn_G[aux].mean()

            knnk_G.append(average_knn)

            ks_G.append(k)
            
    return ks_G, knnk_G

In [132]:
def knn(G):
    
    knn_G = np.zeros(len(G.nodes), dtype = float)
    
    for i in G.nodes:

        aux = nx.average_neighbor_degree(G, nodes = [i])
        knn_G[i] = float(aux[i])

    return knn_G

In [139]:
def clean_graph(G, remove_edges = True):
    
    G = G.to_undirected()
    
    if remove_edges:
        
        G.remove_edges_from(nx.selfloop_edges(G))
    
    G_cc = sorted(nx.connected_components(G), key = len, reverse =True)
    G = G.subgraph(G_cc[0])
    G = nx.convert_node_labels_to_integers(G, first_label = 0 )
    
    return G